In [ ]:
<center><h1>DATAJAM.AI CATS & DOGS - <a style="color:red">HYPERPARAMETERS TUNING</a> -</h1></center>

<img src="https://i.ibb.co/zJyLQ0s/Screen-Shot-2019-07-20-at-22-42-19.png" style="width:300px">

In [1]:
import kerastuner
import  tensorflow as tf
from kerastuner.tuners import RandomSearch
import numpy as np
import cv2
from tqdm import tqdm

In [21]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
#from keras.applications import VGG16


from tensorflow.keras import backend
from tensorflow.compat.v1.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import plot_model


Using TensorFlow backend.


In [2]:
# Load the fashion-mnist pre-shuffled train data and test data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
# we limit the experiment to a few data points
limits = 1000
(x_train, y_train), (x_test, y_test) = (x_train[:limits], y_train[:limits]), (x_test[:limits], y_test[:limits])

In [4]:
# gives us information on the number of classes predicted
categories =  np.unique(y_train).size

In [5]:
# ratio of data used for training 
ratio = 0.8

In [6]:
def represent_picture(x,y):
    an_im = plt.imshow(x, cmap='gray')
    plt.title(y)
    plt.show()
    return(x)

In [7]:
def resize_me(x,size=(224,224),mode=cv2.INTER_CUBIC):
    # resize the image to the network input shape
    res= cv2.resize(x,(224,224),mode)
    return(cv2.merge([res,res,res]))

In [8]:
# we loop through all the images
# resize the image with the function defined above
# save the numpy array "of the list"

l=[]
for el in tqdm(x_train):
    l.append(resize_me(el,size=(224,224),mode=cv2.INTER_CUBIC))
x_train_new = np.array(l)

100%|██████████| 1000/1000 [00:00<00:00, 6843.64it/s]


In [9]:
train_data, train_labels= x_train_new[:int(ratio*x_train.shape[0])],y_train[:int(ratio*x_train.shape[0])]
val_data, val_labels= x_train_new[int(ratio*x_train.shape[0]):],y_train[int(ratio*x_train.shape[0]):]

In [17]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(32, (3, 3), input_shape=(224, 224,3)))
    model.add(layers.Flatten())
    model.add(layers.Dense(10))
    model.add(layers.Dense(units=hp.Range('units',
                                          min_value=32,
                                          max_value=512,
                                          step=32),
                           activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
# optimizing a structure (after downloading from VGG16)
# grandomsearch : 
# on the number of dense layers
# on the dropout values
# on the learning reate values

In [24]:
def build_model(hp):
    categories=10
    size=(224,224,3)

    baseModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=size))

    # construct the head of the model that will be placed on top of the
    # the base model
    headModel = baseModel.output
    headModel = layers.Flatten(name="flatten")(headModel)
    headModel = layers.Dense(units=hp.Range('units',
                                     min_value=32,
                                     max_value=512,
                                     step=32), 
                      activation="relu")(headModel)

    headModel = layers.Dropout(hp.Choice('dropout', 
                                values=[0.1,0.2,0.3,0.4,0.5]))(headModel)
    headModel = layers.Dense(categories, activation="softmax")(headModel)

    # place the head FC model on top of the base model (this will become
    # the actual model we will train)
    model = Model(inputs=baseModel.input, outputs=headModel)
    model.compile(
    optimizer=keras.optimizers.Adam(
        hp.Choice('learning_rate',
                  values=[1e-2, 1e-3, 1e-4])),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
    return model

In [25]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

In [ ]:
tuner.search(train_data, train_labels,
             epochs=5,
             validation_data=(val_data, val_labels))